In [98]:
import pandas as pd
import numpy as np 
pd.options.display.max_colwidth = 100

### Se carga la lista de restaurantes scrappeados:

In [99]:
restaurantes=pd.read_csv("restaurantes_08062020pm.csv")

In [100]:
print("Se tienen: ", len(restaurantes), " restaurantes")

Se tienen:  2448  restaurantes


In [101]:
restaurantes.head()

,Unnamed: 0,Nombre,Categoria_Especifica,Calificacion,Reviews,Direccion,Url
0,0,Versus,$$ • Comida rápida,4.3,(34),"Carrera 13# 8n -39, Quindío 630003 • Más información",https://www.ubereats.com/co/armenia/food-delivery/versus/UhZCYICBSu2cwJoiF5kfHQ
1,1,Don Jacobo Postres y Ponques - Armenia,$ • Postres,4.8,(52),"Cra. 14 No. 9 Norte-69, Av. Bolivar, Armenia, Quindío, Colombia, • Más información",https://www.ubereats.com/co/armenia/food-delivery/don-jacobo-postres-y-ponques-armenia/vhYajc-nS...
2,2,Le Ricette Pizza,$$ • Italiana,NaN,NaN,"630001 Cra. 6 #13n-27 Local, Armenia, Quindío 630004 • Más información",https://www.ubereats.com/co/armenia/food-delivery/le-ricette-pizza/GNCrqZHSTpSS5JYo3Q_h2w
3,3,Versus,$$ • Comida rápida,4.3,(34),"Carrera 13# 8n -39, Quindío 630003 • Más información",https://www.ubereats.com/co/armenia/food-delivery/versus/UhZCYICBSu2cwJoiF5kfHQ
4,4,Don Jacobo Postres y Ponques - Armenia,$ • Postres,4.8,(52),"Cra. 14 No. 9 Norte-69, Av. Bolivar, Armenia, Quindío, Colombia, • Más información",https://www.ubereats.com/co/armenia/food-delivery/don-jacobo-postres-y-ponques-armenia/vhYajc-nS...


### Existen resturantes que figuran en varias categorías generales, y tendrán registros duplicados, se eliminan:

In [102]:
restaurantes.drop("Unnamed: 0" ,axis=1, inplace=True)
restaurantes.drop_duplicates(keep="first",inplace=True)

### Variables:

In [103]:
restaurantes.columns

Index(['Nombre', 'Categoria_Especifica', 'Calificacion', 'Reviews',
       'Direccion', 'Url'],
      dtype='object')

### * NOMBRE: 
Nombre con el cuál se identifica el restaurante en la página. (String)
### * CATEGORIA_ ESPECIFICA:
 Diferentes categorías en las cuales se puede ubicar el restaurante. Puede ser multivaluado. Es necesario hacer una limpieza, ya que los datos vienen sucios. (string)

### * CALIFICACIÓN: 
Puntuación que reporta la página para el restaurante. (float)

### * REVIEWS: 
Número de reviews que presenta la página para el restaurante. Es necesario hacer limpieza, ya que viene con caracteres sucios. (String)

### * Dirección:
 Ubicación que reporta la página para el restaurante. Es necesario hacer limpieza, ya que viene con datos sucios.

### * URL: 
URL exacta con la cuál se identifica el restaurante en la página.


### Se observa el dtypes:

In [104]:
print("Tipos de datos: ") 
restaurantes.dtypes

Tipos de datos: 


Nombre                   object
Categoria_Especifica     object
Calificacion            float64
Reviews                  object
Direccion                object
Url                      object
dtype: object

### Buscar valores faltantes: 

In [105]:
restaurantes.isna().sum()

Nombre                    0
Categoria_Especifica      1
Calificacion            587
Reviews                 587
Direccion                 2
Url                       0
dtype: int64

### CALIFICACIÓN Y REVIEWS FALTANTES:
 La mayor cantidad de los faltantes en los registros, está en calificación y número de reviews. Esto es porque la mayor parte de los restaurantes cuando están cerrados ocultan su calificación y número de reviews. Adicional a eso, cuando un restaurante es nuevo en la página, tampoco reporta calificación y reviews. Es importante correr el algoritmo en el momento del día que la mayor parte de los restaurantes esté abierto.

### RESTO DE VARIABLES FALTANTES:
El resto de variables faltantes en los registros, que son muy pocas, es porque el restaurante no tenía la información disponible.

### LIMPIEZA DE DATOS:

#### * DIRECCIÓN:

In [109]:
restaurantes["Direccion"].head()

0                                  Carrera 13# 8n -39, Quindío 630003
1    Cra. 14 No. 9 Norte-69, Av. Bolivar, Armenia, Quindío, Colombia,
2                630001 Cra. 6 #13n-27 Local, Armenia, Quindío 630004
6                          Cl. 5 Nte. #14–06, Armenia, Quindío 630004
7                           Armenia, Quindío, Colombia, Mall Del Rio,
Name: Direccion, dtype: object

Se observa que tienen el string " • Mas información" en todos lo registros. Se procede a eliminar. Así mismo se borran espacios en blanco al comienzo y final.

In [107]:
restaurantes["Direccion"]=restaurantes["Direccion"].apply(lambda x : str(x).replace("•", "").replace("Más información","").strip()) ##Borrar string y espacios

In [108]:
restaurantes["Direccion"][1] ##Prueba

'Cra. 14 No. 9 Norte-69, Av. Bolivar, Armenia, Quindío, Colombia,'

### * REVIEWS:

In [111]:
restaurantes["Reviews"].value_counts()

(200+)    28
(100+)    27
(13)      25
(17)      21
(50+)     18
          ..
(108)      1
(147)      1
(50)       1
(166)      1
(82)       1
Name: Reviews, Length: 113, dtype: int64

Se nota como hay paréntesis y signos "+" no deseados, pues el interes está únicamente en el valor entero. Se procede a eliminar el signo "+" y los parentesis. Adicional a eso, se hace la conversión a columna numérica.

In [112]:
restaurantes["Reviews"]=restaurantes["Reviews"].apply(lambda x : str(x).replace("+", "").replace("(","").replace(")","").strip()) ##Borrar string y espacios

In [123]:
restaurantes['Reviews'] = restaurantes.Reviews.astype('float').astype("Int64")
restaurantes["Reviews"].head()
restaurantes["Reviews"].dtype

Int64Dtype()

### CATEGORÍA ESPECIFICA:

Puede verse que todas los registros tienen un " \$ " o " \$\$ ".
 No sé sabe con seguridad lo que significa esto, pero parece importante almacenar si tiene uno o dos. Se retirarán dichos signos del registro de categoría, pero se creará otra variable que diga cuantos "$" tenía.

In [135]:
restaurantes['Cantidad_$']=restaurantes["Categoria_Especifica"].apply(lambda x: str(x).count("$"))

In [138]:
restaurantes["Cantidad_$"].head()

0    2
1    1
2    2
6    2
7    2
Name: Cantidad_$, dtype: int64